# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

The dataset collected is related to 17 campaigns that occurred between May 2008 and November 2010, corresponding to a total of 79354 contacts. 

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [37]:
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.dummy import DummyClassifier
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier

In [38]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [39]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [40]:
print(df.duplicated().sum())
#Drop the duplicated values
df = df.drop_duplicates()

12


It appears that there are several features that have the value unknown in them.
It looks like data has been cleaned and unknown's have been added to all columns where a null value was encountered.

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

## Business Objective - Use classification models to predict whether or not the customer will subscribe to a term deposit.

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41176 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41176 non-null  int64  
 1   job             41176 non-null  object 
 2   marital         41176 non-null  object 
 3   education       41176 non-null  object 
 4   default         41176 non-null  object 
 5   housing         41176 non-null  object 
 6   loan            41176 non-null  object 
 7   contact         41176 non-null  object 
 8   month           41176 non-null  object 
 9   day_of_week     41176 non-null  object 
 10  duration        41176 non-null  int64  
 11  campaign        41176 non-null  int64  
 12  pdays           41176 non-null  int64  
 13  previous        41176 non-null  int64  
 14  poutcome        41176 non-null  object 
 15  emp.var.rate    41176 non-null  float64
 16  cons.price.idx  41176 non-null  float64
 17  cons.conf.idx   41176 non-null 

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features (columns 1 - 7), prepare the features and target column for modeling with appropriate encoding and transformations.

In [42]:
#Drop the suplicates
df = df.drop_duplicates()

In [43]:
#Determine if there are any null values
df.isnull().values.any()

False

In [44]:
#Remove outliers

In [45]:
labelEncoder = LabelEncoder()

categorical_vars =['contact','month','day_of_week','poutcome']
for cat_var in categorical_vars:
    df[cat_var]= labelEncoder.fit_transform(df[cat_var]) 

df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [46]:
X = df[['contact','month','day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed']] 

y = df['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [47]:
baseLineModel = DummyClassifier(strategy="stratified") 
baseLineModel.fit(X_train, y_train.ravel())

DummyClassifier(strategy='stratified')

In [48]:
baseLineModel_accuracy = baseLineModel.score(X_train, y_train.ravel())
print("Accuracy of baseline model to beat: {0:.8f}".format(baseLineModel_accuracy))
print()

Accuracy of baseline model to beat: 0.80099226



### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [54]:
logisticRegressionModel = LogisticRegression(C = 0.7, random_state = 42, max_iter = 200)
start_time = time.time()
logisticRegressionModel.fit(X_train, y_train.ravel())
elapsed_time = time.time() - start_time
print(f'{elapsed_time:.2f}s elapsed during training')

train_prediction = logisticRegressionModel.predict(X_train)

test_prediction = logisticRegressionModel.predict(X_test)

0.25s elapsed during training


### Problem 9: Score the Model

What is the accuracy of your model?

In [55]:
logistics_model_train_accuracy = metrics.accuracy_score(y_train, train_prediction)
print("Train Accuracy: {0:.8f}".format(logistics_model_train_accuracy))
logistics_model_test_accuracy = metrics.accuracy_score(y_test, test_prediction)
print("Test Accuracy: {0:.8f}".format(logistics_model_test_accuracy))
print()

Train Accuracy: 0.91142490
Test Accuracy: 0.90682425



### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [51]:
decision_tree = tree.DecisionTreeClassifier()
start_time = time.time()
decision_tree.fit(X_train, y_train.ravel())
elapsed_time = time.time() - start_time
print(f'{elapsed_time:.2f}s elapsed during training')

train_prediction = decision_tree.predict(X_train)
train_accuracy = metrics.accuracy_score(y_train, train_prediction)
print("Train Accuracy: {0:.8f}".format(train_accuracy))

test_prediction = decision_tree.predict(X_test)
test_accuracy = metrics.accuracy_score(y_test, test_prediction)
print("Test Accuracy: {0:.8f}".format(test_accuracy))
print()

0.11s elapsed during training
Train Accuracy: 0.99916733
Test Accuracy: 0.88205294



In [52]:
supportvector_classifier = svm.SVC()
start_time = time.time()
supportvector_classifier.fit(X_train, y_train.ravel())
elapsed_time = time.time() - start_time
print(f'{elapsed_time:.2f}s elapsed during training')

svm_train_prediction = supportvector_classifier.predict(X_train)
svm_train_accuracy = metrics.accuracy_score(y_train, svm_train_prediction)
print("Train Accuracy: {0:.8f}".format(svm_train_accuracy))

svm_test_prediction = supportvector_classifier.predict(X_test)
svm_test_accuracy = metrics.accuracy_score(y_test, svm_test_prediction)
print("Test Accuracy: {0:.8f}".format(svm_test_accuracy))
print()

8.34s elapsed during training
Train Accuracy: 0.89872671
Test Accuracy: 0.89638145



In [53]:
acc = []
acc1 = []
for i in range(1,40):
    neigh = KNeighborsClassifier(weights='distance', n_neighbors=i).fit(X_train,y_train)
    train_yhat = neigh.predict(X_train)
    acc1.append(metrics.accuracy_score(y_train, train_yhat))
    yhat = neigh.predict(X_test)
    acc.append(metrics.accuracy_score(y_test, yhat))
print("Train Accuracy: {0:.8f}".format(max(acc1)))
print("Test Accuracy: {0:.8f}".format(max(acc)),"at K =",acc.index(max(acc)))

Train Accuracy: 0.99916733
Test Accuracy: 0.90609569 at K = 26


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

##### Questions